In [1]:
from DLScript import *

In [2]:
embedding_matrix = np.load('Script pickles/embedding_matrix.pkl', allow_pickle=True)
word2vec_vocab = np.load('Script pickles/word2vec_vocab.pkl', allow_pickle=True)
dl_obj = DLScript('Script pickles/LSTM_3projects_3Linearlayers_10epochs.pth', word2vec_vocab, embedding_matrix, embedding_matrix.shape, 4000, 2000)

UC_file = ''
with open('Dataset/errai_dataset/test_UC/ERRAI-259.txt', 'r') as file:
    UC_file = file.read()

dl_obj.UseCaseSourceFileScript('Dataset/errai_dataset/test_CC/588.java', UC_file)

# print(word2vec_vocab)

4


100%|██████████| 1/1 [00:00<00:00,  3.50it/s]

tensor([1, 1, 1, 0])
positive_labels 3
75.0
